In [ ]:
!pip install langchain transformers
!pip install sentence-transformers
!pip install chromadb --user
!pip install replicate

In [1]:
import os
import sys
from langchain.llms import Replicate
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader

In [2]:
os.environ['REPLICATE_API_TOKEN'] = "INSERT YOUR KEY HERE"
# https://replicate.com/

In [3]:
loader = CSVLoader(file_path="congestionpoints.csv")
documents = loader.load()

In [4]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [5]:
embeddings = HuggingFaceEmbeddings()

In [6]:
llm = Replicate(
    model="a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5",
    model_kwargs={"temperature": 0.75, "max_length": 3000}
)

In [7]:
from langchain.vectorstores import Chroma

In [8]:
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory="./store"
)

In [9]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectordb.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True
)


In [ ]:
chat_history = []
while True:
    query = input('Prompt: ')
    '''
        As a start, you can query this - 
        Analyse vehicle centroid motion and traffic motion standard deviation to identify congestion hotspots on the map. Provide recommendations for structural changes to optimise traffic flow and reduce congestion occurence, specifying the coordinates for implementation.
    '''
    if query == "exit" or query == "quit" or query == "q":
        print('Exiting')
        sys.exit()
    result = qa_chain({'question': query, 'chat_history': chat_history})
    print('Query: '+query+'\n')
    print('Answer: ' + result['answer'] + '\n')
    chat_history.append((query, result['answer']))

Prompt: Analyse vehicle centroid motion and traffic motion standard deviation to identify congestion hotspots on the map. Provide recommendations for structural changes to optimise traffic flow and reduce congestion occurence, specifying the coordinates for implementation.


C:\Users\bharg\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Query: Analyse vehicle centroid motion and traffic motion standard deviation to identify congestion hotspots on the map. Provide recommendations for structural changes to optimise traffic flow and reduce congestion occurence, specifying the coordinates for implementation.

Answer: Congestion hotspots are identified based on high standard deviation of vehicle coordinates, indicating high variability in traffic motion. To address this, recommendations for structural changes include:
1. Widening road lanes at coordinates (25.84519577059521, 66.4308268562001) to increase capacity and reduce bottlenecks.
2. Adding turn lanes at coordinates (20.20703828940581, 28.73160731540289) to improve flow and reduce merging conflicts.

Please use the information provided to answer the question at the end.

